In [1]:
import numpy as np
from scipy.stats import mode
from pbil import pbil
import multiprocessing
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
from numpy.random import random as srand
import matplotlib.pyplot as plt

/pio/os/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


Populating the interactive namespace from numpy and matplotlib


In [2]:
data = np.loadtxt('../data/ImageRawReduced.txt')
targets = np.loadtxt('../data/ImageExpertReduced.txt')
rules = np.loadtxt('../data/ClassificationRules.txt')
data.shape, targets.shape, rules.shape

((3, 9350), (9350,), (266, 9350))

In [3]:
def draw_plots(scores_history, props_history, problem_title=None):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
    if problem_title:
        fig.suptitle(problem_title, fontsize=20)
    ax1.plot(np.arange(len(scores_history)), scores_history)
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Score')
    ax1.set_title('Progress')

    
    for x in np.array(props_history).T:
        ax2.plot(np.arange(x.size), x)
        ax2.set_title('Propabilities')
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Propability')
    plt.show()

In [42]:
import pickle
G = 0
def process_instance(rules_mask):
    if not(
    try:
        temp = (mode(
            rules[rules_mask],
            axis=0
        )[0] == targets).astype(np.int).sum()
    return temp
    
def f(pop):
    res = np.zeros((pop.shape[0], targets.size))
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        res = pool.map(
            process_instance,
            (process_instance(x) for x in pop)
        )
    return np.array(res)
#     return np.array([process_instance(x) for x in pop])

In [5]:
# %%time
# res = pbil(
#     f,
#     vec_size=rules.shape[0],
#     pop_size=100,
#     iterations=4000,
#     theta1=0.1,
#     theta2=0.07,
#     theta3=0.05,
# )
# draw_plots(res[1], res[2], 'Spectral')

In [26]:
def pbil(F, d, N, th1, th2, th3, steps=500):
    
    def random_pop(p, d, N):
        population = srand((d, N))
        return population < p
    
    def get_score(scores):
        return np.max(scores), np.mean(scores), np.min(scores)
    
    
    plot_probs = []
    plot_scores = []
    
    p = np.ones((d, 1)) / 2
    population = random_pop(p, d, N)
    result = F(population)
    
    plot_probs.append(p)
    plot_scores.append(get_score(result))
    
    for _ in range(steps - 1):
        best = population[:, [np.argmax(result)]]
        p = p * (1 - th1) + best * th1
        
        if_change = srand((d,1)) < th2
        p = p * (1 - if_change) + (
            (if_change) * (
                p * (1 - th3)
                + (srand((d,1)) < 0.5) * th3
            )
        )
        
        population = random_pop(p, d, N)
        result = F(population)
        
        plot_probs.append(p)
        plot_scores.append(get_score(result))
    
    return np.array(plot_scores), np.array(plot_probs)
        
def plot_results(scores, probs, title=""):
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(title)
    for i in range(scores.shape[1]):
        axs[0].plot(range(scores.shape[0]), scores[:, i])
    
    for i in range(probs.shape[1]):
        axs[1].plot(range(probs.shape[0]), probs[:,i,0])
    plt.show()

In [43]:
%%time
plot_results(
    *pbil(
        f,
        d=50,
        N=rules.shape[0],
        th1=0.1,
        th2=0.07,
        th3=0.05,
        steps=100
    ),
    f"N: {rules.shape[0]}"
)

/pio/os/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  import sys
/pio/os/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


TypeError: file must have a 'write' attribute

In [41]:
with open('templog') as file:
    x = pickle.load(file)

TypeError: a bytes-like object is required, not 'str'

In [49]:
G.sum()

0